In [ ]:
!pip install wget

In [ ]:
import wget
import os
import tarfile

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from keras.layers import SimpleRNN,GRU,LSTM,Embedding,Input,Dense

In [ ]:
filename = 'dakshina_dataset_v1.0'
url = 'https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar'
if not os.path.exists(filename+'.tar') and not os.path.exists(filename):
    filename_tar = wget.download(url)
    file = tarfile.open(filename_tar)
    print('\nExtracting files ....')
    file.extractall()
    file.close()
    print('Done')
    os.remove(filename_tar)
elif not os.path.exists(filename):
    filename_tar = filename + '.tar'
    file = tarfile.open(filename_tar)
    print('\nExtracting files ....')
    file.extractall()
    file.close()
    print('Done')
    os.remove(filename_tar)

In [ ]:
lang = 'bn'
train_path =  filename+f"/{lang}/lexicons/{lang}.translit.sampled.train.tsv"
val_path = filename+f"/{lang}/lexicons/{lang}.translit.sampled.dev.tsv"
test_path = filename+f"/{lang}/lexicons/{lang}.translit.sampled.test.tsv"

In [ ]:
def read_data(path):
    df = pd.read_csv(path,header=None,sep='\t')
    df.fillna("NaN",inplace=True)
    input_texts,target_texts = df[1].to_list(),df[0].to_list()
    return input_texts,target_texts

In [ ]:
def parse_text(texts):
    characters = set()
    for text in texts:
        for c in text:
            if c not in characters:
                characters.add(c)
    characters.add(' ')
    return sorted(list(characters))

In [ ]:
def start_end_pad(texts):
    for i in range(len(texts)):
        texts[i] = "\t" + texts[i] + "\n"
    return texts

In [ ]:
train_input_texts,train_target_texts = read_data(train_path)
val_input_texts,val_target_texts = read_data(val_path)
test_input_texts,test_target_texts = read_data(test_path)

In [ ]:
train_target_texts = start_end_pad(train_target_texts)
val_target_texts = start_end_pad(val_target_texts)
test_target_texts = start_end_pad(test_target_texts)

In [ ]:
config_ = {
    "learning_rate": 1e-3,                                      # Learning rate in gradient descent
    "epochs": 10,                                               # Number of epochs to train the model   
    "optimizer": 'adam',                                        # Gradient descent algorithm used for the parameter updation
    "batch_size": 64,                                           # Batch size used for the optimizer
    "loss_function": 'categorical_crossentropy',                # Loss function used in the optimizer                                                                      # Name of dataset
    "input_embedding_size": 64,                                        # Size of input embedding layer
    "num_enc_layers": 1,                                         # Number of layers in the encoder
    "num_dec_layers": 1,                                         # Number of layers in the decoder
    "hidden_layer_size": 128,                                      # Size of hidden layer
    "dropout" : 0.20, 
    'r_dropout':0.20,# Value of dropout used in the normal and recurrent dropout
    "cell_type": 'LSTM',                                         # Type of cell used in the encoder and decoder ('RNN' or 'GRU' or 'LSTM')
    "beam_width": 5                                           # Beam width used in beam decoder                                         # Whether or not attention is used
}

In [ ]:
def enc_dec_tokens(train_input_texts,train_target_texts,val_input_texts,val_target_texts):
    
    input_characters = parse_text(train_input_texts + val_input_texts)
    target_characters = parse_text(train_target_texts + val_target_texts)
    num_encoder_tokens = len(input_characters)
    num_decoder_tokens = len(target_characters)
    max_encoder_seq_length = max([len(txt) for txt in train_input_texts + val_input_texts])
    max_decoder_seq_length = max([len(txt) for txt in train_target_texts + val_target_texts])

    print("Number of training samples:", len(train_input_texts))
    print("Number of validation samples:", len(val_input_texts))
    print("Number of unique input tokens:", num_encoder_tokens)
    print("Number of unique output tokens:", num_decoder_tokens)
    print("Max sequence length for inputs:", max_encoder_seq_length)
    print("Max sequence length for outputs:", max_decoder_seq_length)
    
    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
    
    return input_token_index,target_token_index,max_encoder_seq_length,max_decoder_seq_length,num_encoder_tokens,num_decoder_tokens

In [ ]:
def data_processing(input_texts,enc_length,input_token_index,num_encoder_tokens, target_texts,dec_length,target_token_index,num_decoder_tokens):
    encoder_input_data = np.zeros(
        (len(input_texts), enc_length), dtype="float32"
    )
    decoder_input_data = np.zeros(
            (len(input_texts), dec_length), dtype="float32"
        )
    decoder_target_data = np.zeros(
            (len(input_texts), dec_length, num_decoder_tokens), dtype="float32"
        )

    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        
        for t, char in enumerate(input_text):
            encoder_input_data[i, t] = input_token_index[char]
        encoder_input_data[i, t + 1 :] = input_token_index[' ']
        
        for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t] = target_token_index[char]
            if t > 0:
                    # decoder_target_data will be ahead by one timestep
                    # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
        decoder_input_data[i, t + 1 :] = target_token_index[' ']
        decoder_target_data[i, t:, target_token_index[' ']] = 1.0
    return encoder_input_data,decoder_input_data,decoder_target_data

In [ ]:
input_token_index,target_token_index,max_encoder_seq_length,max_decoder_seq_length,num_encoder_tokens,num_decoder_tokens = enc_dec_tokens(train_input_texts,train_target_texts,val_input_texts,val_target_texts)
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
train_encoder_input,train_decoder_input,train_decoder_target = data_processing(train_input_texts,max_encoder_seq_length,input_token_index,num_encoder_tokens, train_target_texts,max_decoder_seq_length,target_token_index,num_decoder_tokens)
val_encoder_input,val_decoder_input,val_decoder_target = data_processing(val_input_texts,max_encoder_seq_length,input_token_index,num_encoder_tokens, val_target_texts,max_decoder_seq_length,target_token_index,num_decoder_tokens)
test_encoder_input,test_decoder_input,test_decoder_target = data_processing(test_input_texts,max_encoder_seq_length,input_token_index,num_encoder_tokens, test_target_texts,max_decoder_seq_length,target_token_index,num_decoder_tokens)

In [ ]:
from keras.layers import Layer
import keras.backend as K

class Attention(Layer):
    """
    This Attention layer class code is used from : https://github.com/thushv89/attention_keras/blob/master/src/layers/attention.py
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
    """

    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(Attention, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            # (batch_size, decoder_timesteps, decoder_hid_layer_size)
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            # (batch_size, decoder_timesteps, encoder_timesteps)
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [ ]:
def make_model(num_encoder_tokens,num_decoder_tokens,input_embedding_size=32,num_enc_layers=1,num_dec_layers=1,hidden_layer_size=64,cell_type='LSTM',dropout=0,r_dropout=0,cell_activation='tanh'):
    cell = {
        'RNN':SimpleRNN,
        'LSTM':LSTM,
        'GRU':GRU
    }
    encoder_input = Input(shape=(None,),name='input_1')
    encoder_input_embedding = Embedding(num_encoder_tokens,input_embedding_size,name='embedding_1')(encoder_input)
    
    encoder_sequences, *encoder_state = cell[cell_type](hidden_layer_size,activation=cell_activation,return_sequences=True,return_state=True,dropout=dropout,recurrent_dropout=r_dropout,name="encoder_1")(encoder_input_embedding)
    
    for i in range(1,num_enc_layers):
        encoder_sequences, *encoder_state = cell[cell_type](hidden_layer_size,activation=cell_activation,return_sequences=True,return_state=True,dropout=dropout,recurrent_dropout=r_dropout,name=f"encoder_{i+1}")(encoder_sequences)
        
    decoder_input = Input(shape=(None,),name='input_2')
    decoder_input_embedding = Embedding(num_decoder_tokens,input_embedding_size,name='embedding_2')(decoder_input)
    
    decoder_sequences = decoder_input_embedding
    for i in range(num_dec_layers-1):
        decoder_sequences, *decoder_state = cell[cell_type](hidden_layer_size,activation=cell_activation,return_sequences=True,return_state=True,dropout=dropout,recurrent_dropout=r_dropout,name=f"decoder_{i+1}")(decoder_sequences ,initial_state=encoder_state)
    decoder_sequences, *decoder_state = cell[cell_type](hidden_layer_size,activation=cell_activation,return_sequences=True,return_state=True,dropout=dropout,recurrent_dropout=r_dropout,name="decoder_1")(decoder_input_embedding ,initial_state=encoder_state)
    
    attention_out,attention_scores = Attention(name="attention_1")([encoder_sequences,decoder_sequences])
    
    dense_concat_input = keras.layers.Concatenate(axis=-1,name='concat_layer_1')([decoder_sequences,attention_out])
    decoder_dense = Dense(num_decoder_tokens,activation="softmax",name="dense_1")(dense_concat_input)
    
    model = keras.Model([encoder_input,decoder_input],decoder_dense)
    model.summary()
    return model

In [ ]:
def make_inference_model(model):
    # Calculating number of layers in encoder and decoder
    num_enc_layers, num_dec_layers = 0, 0
    for layer in model.layers:
        num_enc_layers += layer.name.startswith('encoder')
        num_dec_layers += layer.name.startswith('decoder')

    # Encoder input
    encoder_input = model.input[0]      # Input_1
    # Encoder cell final layer
    encoder_cell = model.get_layer("encoder_"+str(num_enc_layers))
    encoder_type = encoder_cell.__class__.__name__
    encoder_sequences, *encoder_state = encoder_cell.output
    # Encoder model
    encoder_model = keras.Model(encoder_input, encoder_state)

    # Decoder input
    decoder_input = model.input[1]      # Input_2
    decoder_input_embedding = model.get_layer("embedding_2")(decoder_input)
    decoder_sequences = decoder_input_embedding
    # Inputs to decoder layers' initial states
    decoder_states, decoder_state_inputs = [], []
    for i in range(1, num_dec_layers+1):
        if encoder_type == 'LSTM':
            decoder_state_input = [Input(shape=(encoder_state[0].shape[1],), name="input_"+str(2*i+1)), 
                                   Input(shape=(encoder_state[1].shape[1],), name="input_"+str(2*i+2))]
        else:
            decoder_state_input = [Input(shape=(encoder_state[0].shape[1],), name="input_"+str(i+2))]

        decoder_cell = model.get_layer("decoder_"+str(i))
        decoder_sequences, *decoder_state = decoder_cell(decoder_sequences, initial_state=decoder_state_input)
        decoder_states += decoder_state
        decoder_state_inputs += decoder_state_input
    
    attention_out,attention_scores = model.get_layer("attention_1")([encoder_sequences,decoder_sequences])
    
    dense_concat_input = keras.layers.Concatenate(axis=-1,name='concat_layer_1')([decoder_sequences,attention_out])
    # Softmax FC layer
    decoder_dense = model.get_layer("dense_1")
    decoder_dense_output = decoder_dense(dense_concat_input)

    # Decoder model
    decoder_model = keras.Model(
        [encoder_input,decoder_input] + decoder_state_inputs, [attention_scores,decoder_dense_output] + decoder_states
    )

    return encoder_model, decoder_model, num_enc_layers, num_dec_layers


def num_to_word(num_encoded, token_index, reverse_char_index = None):
    # Function to return the predictions after cutting the END_CHAR and BLANK_CHAR s at the end.
    # If char_dec == None, the predictions are in the form of decoded string, otherwise as list of integers
    num_samples = len(num_encoded) if type(num_encoded) is list else num_encoded.shape[0]
    predicted_words = ['' for t in range(num_samples)]
    for i, encode in enumerate(num_encoded):
        for l in encode:
            # Stop word : '\n'
            if l == token_index['\n']:
                break
            predicted_words[i] += reverse_char_index[l] if reverse_char_index is not None else str(l)
    
    return predicted_words

In [ ]:
def beam_decoder_util(model,input_sequences,max_decoder_seq_length,B=1,target_sequences=None,start_char=0,batch_size=64):
    encoder_model,decoder_model,num_enc_layers,num_dec_layers=make_inference_model(model)
    encoder_output = encoder_model.predict(input_sequences,batch_size=batch_size)
    encoder_output = encoder_output if type(encoder_output) is list else [encoder_output]
    
    num_samples = input_sequences.shape[0]
    
    outputs_fn = np.zeros((num_samples,B,max_decoder_seq_length),dtype=np.int32)
    
    errors_fn = np.zeros((num_samples,B))
    
    decoder_b_inputs = np.zeros((num_samples,1,1))
    decoder_b_inputs[:, :, 0] = start_char
    
    decoder_b_out = [[(0, [])] for t in range(num_samples)]
    errors = [[0] for t in range(num_samples)]
    
    states = [encoder_output*num_dec_layers]
    attn_b_scores = [[None] for t in range(num_samples)]
    
    for idx in range(max_decoder_seq_length):
        all_b_beams = [[] for t in range(num_samples)]
        all_decoder_states = [[] for t in range(num_samples)]
        all_errors = [[] for t in range(num_samples)]
        all_attn_scores = [[] for t in range(num_samples)]
        for b in range(len(decoder_b_out[0])):
            attn_scores,decoder_output, *decoder_states = decoder_model.predict([input_sequences,decoder_b_inputs[:,b]] + states[b],batch_size=batch_size)
            top_b = np.argsort(decoder_output[:,-1,:],axis=-1)[:,-B:]
            for n in range(num_samples):
                all_b_beams[n]+= [(decoder_b_out[n][b][0] + np.log(decoder_output[n, -1, top_b[n][i]]),decoder_b_out[n][b][1] + [top_b[n][i]]) for i in range(B)]
                all_attn_scores[n] += [attn_scores[n]]*B if attn_b_scores[n][b] is None else [np.concatenate((attn_b_scores[n][b],attn_scores[n]),axis=0)]*B
                if target_sequences is not None:
                    all_errors[n] += [errors[n][b] - np.log(decoder_output[n,-1,target_sequences[n,idx]])]*B
                all_decoder_states[n] += [[decoder_state[n:n+1] for decoder_state in decoder_states]] * B
        sorted_index = list(range(len(all_b_beams[0])))
        sorted_index = [sorted(sorted_index,key = lambda ix: all_b_beams[n][ix][0])[-B:][::-1] for n in range(num_samples)]
        decoder_b_out = [[all_b_beams[n][index] for index in sorted_index[n]] for n in range(num_samples)]
        
        decoder_b_inputs = np.array([[all_b_beams[n][index][1][-1] for index in sorted_index[n]] for n in range(num_samples)])
        
        states = [all_decoder_states[0][index] for index in sorted_index[0]]
        
        for n in range(1,num_samples):
            states = [[np.concatenate((states[i][j],all_decoder_states[n][index][j])) for j in range(len(all_decoder_states[n][index]))] for i,index in  enumerate(sorted_index[n])]
        attn_b_scores = [[all_attn_scores[n][index] for index in sorted_index[n]] for n in range(num_samples)]    
        if target_sequences is not None:
            errors = [[all_errors[n][index] for index in sorted_index[n]] for n in range(num_samples)]
    outputs_fn = np.array([[decoder_b_out[n][i][1] for i in range(B)] for n in range(num_samples)])
    if target_sequences is not None:
        errors_fn = np.array(errors)/max_decoder_seq_length
    return outputs_fn,errors_fn,np.array(states),np.array(attn_b_scores)
def calc_metrics(b_outputs, target_sequences,token_index,reverse_char_index,b_errors=None,exact_word=True,display=False):
    matches = np.mean(b_outputs == target_sequences.reshape((target_sequences.shape[0],1,target_sequences.shape[1])),axis=-1)
    best_b = np.argmax(matches,axis=-1)
    best_index = (tuple(range(best_b.shape[0])),tuple(best_b))
    accuracy = np.mean(matches[best_index])
    b_predictions = list()
    loss = None
    if b_errors is not None:
        loss = np.mean(b_errors[best_index])
    if exact_word:
        equal = [0] * b_outputs.shape[0]
        true_out = num_to_word(target_sequences,token_index,reverse_char_index)
        for b in range(b_outputs.shape[1]):
            pred_out = num_to_word(b_outputs[:,b], token_index,reverse_char_index)
            equal = [equal[i] or (pred_out[i] == true_out[i]) for i in range(b_outputs.shape[0])]
            if display==True:
                b_predictions.append(pred_out)
        exact_accuracy = np.mean(equal)
        if display==True:
            return accuracy,exact_accuracy,loss,true_out,b_predictions
        return accuracy,exact_accuracy,loss
    return accuracy,loss
def beam_decoder(model,input_sequences,target_sequences_onehot,max_decoder_seq_length,token_index,reverse_char_index,B=1,model_batch_size=64,infer_batch_size=512,exact_word=True,return_outputs=False,return_states=False,return_attention=False,display=False):
    target_sequences = np.argmax(target_sequences_onehot,axis=-1)
    b_outputs,b_errors,b_states,b_attention=None,None,None,None
    for i in range(0,input_sequences.shape[0],infer_batch_size):
        tmp_b_outputs,tmp_b_errors,tmp_b_states,tmp_b_attention = beam_decoder_util(model,input_sequences[i:i+infer_batch_size],max_decoder_seq_length,B,target_sequences[i:i+infer_batch_size],token_index['\t'],model_batch_size)
        
        if b_errors is None:
            b_outputs,b_errors,b_states,b_attention = tmp_b_outputs,tmp_b_errors,tmp_b_states,tmp_b_attention
        else:
            b_outputs = np.concatenate((b_outputs,tmp_b_outputs))
            b_errors = np.concatenate((b_errors,tmp_b_errors))
            b_states = np.concatenate((b_states,tmp_b_states),axis=2)
            b_attention = np.concatenate((b_attention,tmp_b_attention))
    return_elements = []
    if return_outputs:
        return_elements += [b_outputs]
    if return_states:
        return_elements += [b_states]
    if return_attention:
        return_elements += [b_attention]
    if len(return_elements) > 0:
        return calc_metrics(b_outputs,target_sequences,token_index,reverse_char_index,b_errors,exact_word,display) + tuple(return_elements)
    return calc_metrics(b_outputs,target_sequences,target_token_index,reverse_char_index,b_errors,exact_word,display)

In [ ]:
# model = make_model(num_encoder_tokens,num_decoder_tokens)

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from tensorflow.keras.metrics import categorical_crossentropy,sparse_categorical_crossentropy
from tensorflow.keras.losses import SparseCategoricalCrossentropy,CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
def model_train_util(config):
    model = make_model(num_encoder_tokens,num_decoder_tokens,config['input_embedding_size'],config['num_enc_layers'],config['num_dec_layers'],config['hidden_layer_size'],config['cell_type'],config['dropout'],config['r_dropout'])
    optimizer = config['optimizer']
    if config['loss_function'] == 'categorical_crossentropy':
        loss_fn = CategoricalCrossentropy
    else:
        loss_fn = SparseCategoricalCrossentropy
    if optimizer == 'adam':
        model.compile(optimizer = Adam(learning_rate=config['learning_rate']), loss = loss_fn(), metrics = ['accuracy'])
    elif optimizer == 'momentum':
        model.compile(optimizer = SGD(learning_rate=config['learning_rate'], momentum = 0.9), loss = loss_fn(), metrics = ['accuracy'])
    elif optimizer == 'rmsprop':
        model.compile(optimizer = RMSprop(learning_rate=config['learning_rate']), loss = loss_fn(), metrics = ['accuracy'])
    elif optimizer == 'nesterov':
        model.compile(optimizer = SGD(learning_rate=config['learning_rate'], momentum = 0.9, nesterov = True), loss = loss_fn(), metrics = ['accuracy'])
    elif optimizer == 'nadam':
        model.compile(optimizer = Nadam(learning_rate=config['learning_rate']), loss = loss_fn(), metrics = ['accuracy'])
    else:
        model.compile(optimizer = SGD(learning_rate=config['learning_rate']), loss = loss_fn(), metrics = ['accuracy'])
    
    return model

In [ ]:
class customCallback(keras.callbacks.Callback):
     # Custom class to provide callback after each epoch of training to calculate custom metrics for validation set with beam decoder
    def __init__(self, val_enc_input, val_dec_target, beam_width=1, batch_size=64):
        self.beam_width = beam_width
        self.validation_input = val_enc_input
        self.validation_target = val_dec_target
        self.batch_size = batch_size

    def on_epoch_end(self, epoch, logs):
        val_accuracy, val_exact_accuracy, val_loss = beam_decoder(self.model, self.validation_input, self.validation_target, max_decoder_seq_length, 
                                                                  target_token_index, reverse_target_char_index, self.beam_width, self.batch_size)

        # Log them to reflect in WANDB callback and EarlyStopping
        logs["val_accuracy"] = val_accuracy
        logs["val_exact_accuracy"] = val_exact_accuracy
        logs["val_loss"] = val_loss             # Validation loss calculates categorical cross entropy loss

        print("— val_loss: {:.3f} — val_accuracy: {:.3f} — val_exact_accuracy: {:.5f}".format(val_loss, val_accuracy, val_exact_accuracy))

In [ ]:
# model,history = model_train_util(config_,[train_encoder_input,train_decoder_input],train_decoder_target,[val_encoder_input,val_decoder_input],val_decoder_target)
# model.save("s2s")

In [ ]:
# Installing and logging into WANDB
!pip install --upgrade wandb
!wandb login b44266d937596fcef83bedbe7330d6cee108a277

import wandb
from wandb.keras import WandbCallback

In [ ]:
def model_train(config,iswandb=False):
    wid = None
    if iswandb:
        wid = wandb.util.generate_id()
        run = wandb.init(id = wid, project="cs6910_assignment_3", entity="dlstack", reinit=True, config=config)
        wandb.run.name = f"ies_{config['input_embedding_size']}_nenc_{config['num_enc_layers']}_ndec_{config['num_dec_layers']}_cell_{config['cell_type']}_drop_{config['dropout']}_rdrop{config['r_dropout']}"
        wandb.run.name += f"_hs_{config['hidden_layer_size']}_B_{config['beam_width']}_attn"
        wandb.run.save()
        print(wandb.run.name)

    model = model_train_util(config)
    if iswandb:
        call_list = [customCallback(val_encoder_input,val_decoder_target,beam_width=config['beam_width'],batch_size=config['batch_size']),WandbCallback(monitor='val_accuracy'),EarlyStopping(monitor='val_accuracy',patience=4)]
    else:
        call_list = [customCallback(val_encoder_input,val_decoder_target,beam_width=config['beam_width'],batch_size=config['batch_size']),EarlyStopping(monitor='val_accuracy',patience=4)]
    history = model.fit(
        [train_encoder_input,train_decoder_input],
        train_decoder_target,
        batch_size=config['batch_size'],
        verbose = 1,
        epochs=config['epochs'],
        callbacks = call_list
    )    
    if iswandb:
        run.finish()

    return model, history,config, wid

In [ ]:
def wandb_sweep():
    # Wrapper function to call the model_train() function for sweeping with different hyperparameters

    # Initialize a new wandb run
    run = wandb.init(config=config_, reinit=True)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    wandb.run.name = f'ies_{config.input_embedding_size}_nenc_{config.num_enc_layers}_ndec_{config.num_dec_layers}_cell_{config.cell_type}_drop_{config.dropout}_rdrop_{config.r_dropout}'
    wandb.run.name += f'_hs_{config.hidden_layer_size}_B_{config.beam_width}_attn'
    wandb.run.save()
    print(wandb.run.name)

    model, *_ = model_train(config, iswandb=True)
    run.finish()

In [ ]:
# Hyperparameter choices to sweep 
sweep_config_1 = {
    'name': 'RNNs2s_attn',
    'method': 'bayes',                   # Possible search : grid, random, bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs':{
            'values':[10,15,20]
        },
        'learning_rate':{
            'values':[0.001,0.0001,0.005]
        },
        'optimizer':{
            'value':'adam'
        },
        'loss_function':{
          'value':'categorical_crossentropy'  
        },
        'input_embedding_size': {
            'values': [64,128,256]
        },
        'num_enc_layers': {
            'values': [2,3]
        },
        'num_dec_layers': {
            'values': [3,5]
        },
        'hidden_layer_size': {
            'values': [256,512,768]
        },
        'cell_type': {
            'values': ['GRU']
        },
        'dropout' :{
            'values': [0.20,0.30]
        },
        'r_dropout': {
            'values': [0.20,0.30]
        },
        'beam_width': {
            'values': [1,3,5]
        },
        'batch_size':{
            'values':[128,256]
        }
    }
}
sweep_id = wandb.sweep(sweep_config_1,entity='dlstack',project='cs6910_assignment_3')
wandb.agent(sweep_id,lambda:wandb_sweep(),count=10)

In [ ]:
# Hyperparameter choices to sweep 
sweep_config_1 = {
    'name': 'RNNs2s_attn',
    'method': 'bayes',                   # Possible search : grid, random, bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs':{
            'values':[10,15,20]
        },
        'learning_rate':{
            'values':[0.001,0.0001,0.005]
        },
        'optimizer':{
            'value':'adam'
        },
        'loss_function':{
          'value':'categorical_crossentropy'  
        },
        'input_embedding_size': {
            'values': [64,128,256]
        },
        'num_enc_layers': {
            'values': [2,3]
        },
        'num_dec_layers': {
            'values': [3,5]
        },
        'hidden_layer_size': {
            'values': [256,512,768]
        },
        'cell_type': {
            'values': ['GRU']
        },
        'dropout' :{
            'values': [0.20,0.30]
        },
        'r_dropout': {
            'values': [0.20,0.30]
        },
        'beam_width': {
            'values': [1,3,5]
        },
        'batch_size':{
            'values':[128,256]
        }
    }
}
sweep_id = wandb.sweep(sweep_config_1,entity='dlstack',project='cs6910_assignment_3')
wandb.agent(sweep_id,lambda:wandb_sweep(),count=10)